In [2]:
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt
stock = fetch_openml(name='stock')
X, y = stock.data, stock.target


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer, mean_squared_error


